In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('mMarFla1.HiC.combined.decontam.20240513.hificovg.pretext.agp_2',
                 sep='\t',
                 comment="#",
                 names = [
                    "Object", "Object_begin", "Object_end", "Part_number",
                    "Component_type", "Component_id", "Component_begin",
                    "Component_end", "Orientation","mod","paint","hap","ex1","ex2"
                    ])

In [6]:
len(df)

52

In [4]:
df = df[(df['Object']=='Scaffold_1') | (df['Object']=='Scaffold_2') | (df['Object']=='Scaffold_3') | (df['Object']=='Scaffold_4')]

In [7]:
len(df)

52

In [8]:
# Concatenate
haplotype = []
for _, row in df.iterrows():
    if pd.notna(row['hap']):
        haplotype.append(f"Super_{hap_counters[current_hap] - 1}_{current_hap}")

In [9]:
df.paint.unique()

array(['Hap_1', nan, 'Hap_2', 'Unloc'], dtype=object)

In [10]:
# Create a new column with Super IDs
hap_counters = {'Hap_1': 1, 'Hap_2': 1}
unloc_counter = 1
current_hap = None
group_ids = []

# Iterate over rows to assign groupID
for _, row in df.iterrows():
    if row["Orientation"] == "proximity_ligation":
        group_ids.append(group_id)
        continue

    paint_value = row["paint"]

    if paint_value in hap_counters:
        # New group for this haplotype
        current_hap = paint_value
        group_id = f"Super_{hap_counters[current_hap]}_{paint_value}"
        hap_counters[current_hap] += 1  # Increment the counter for the haplotype
    elif paint_value == "Unloc" and current_hap:
        # "Unloc" case, no increment in main hap counter
        group_id = f"Super_{hap_counters[current_hap] - 1}_{current_hap}_Unloc_{unloc_counter}"
        unloc_counter += 1  # Increment the Unloc counter within the group
    elif current_hap:
        # Continuation of the current group without new haplotype value
        group_id = f"Super_{hap_counters[current_hap] - 1}_{current_hap}"
        unloc_counter = 1  # Reset Unloc counter if no "Unloc" value

    group_ids.append(group_id)

# Assign the groupID column to the dataframe
df["groupID"] = group_ids

In [11]:
df.head()

,Object,Object_begin,Object_end,Part_number,Component_type,Component_id,Component_begin,Component_end,Orientation,mod,paint,hap,ex1,ex2,groupID
0,Scaffold_1,1,163714215,1,W,H1.scaffold_1,1,163714215,+,Painted,Hap_1,NaN,NaN,NaN,Super_1_Hap_1
1,Scaffold_1,163714216,163714315,2,U,100,scaffold,yes,proximity_ligation,NaN,NaN,NaN,NaN,NaN,Super_1_Hap_1
2,Scaffold_1,163714316,170187104,3,W,H1.scaffold_1,163714216,170187004,-,Painted,NaN,NaN,NaN,NaN,Super_1_Hap_1
3,Scaffold_1,170187105,170187204,4,U,100,scaffold,yes,proximity_ligation,NaN,NaN,NaN,NaN,NaN,Super_1_Hap_1
4,Scaffold_1,170187205,220706538,5,W,H1.scaffold_1,170187005,220706338,+,Painted,NaN,NaN,NaN,NaN,Super_1_Hap_1


In [12]:
import os

# -------------------------------
# Part 1: Generate a new AGP file (new_dummy.agp)
# -------------------------------

agp_filename = "new_dummy.agp"

with open(agp_filename, "w") as f:
    # Write a header comment line with column titles (optional for clarity)
    header_titles = "# Object\tObject_Begin\tObject_End\tPart_Number\tComponent_Type\tComponent_ID\tComponent_Begin\tComponent_End\tOrientation\tComment\tExtra_info\tCol12\tCol13\tCol14\tGroup ID"
    f.write(header_titles + "\n")

    # --- Head Records (10 records) ---
    # Define the original head records (each with 11 columns)
    head_records = [
        # Record 1: set col2=1, col3=10
        ["Scaffold_1", "1", "10", "1", "W", "H1.scaffold_1", "1", "10", "+", "Painted", "Hap_1"],
        # Record 2: dummy values (e.g., 11-20)
        ["Scaffold_1", "11", "20", "2", "U", "100", "scaffold", "yes", "proximity_ligation", "", ""],
        # Record 3: set col2=11, col3=20
        ["Scaffold_1", "11", "20", "3", "W", "H1.scaffold_1", "11", "20", "-", "Painted", ""],
        # Record 4: dummy values (e.g., 21-30)
        ["Scaffold_1", "21", "30", "4", "U", "100", "scaffold", "yes", "proximity_ligation", "", ""],
        # Record 5: set col2=21, col3=30
        ["Scaffold_1", "21", "30", "5", "W", "H1.scaffold_1", "21", "30", "+", "Painted", ""],
        # Record 6: set col2=1, col3=10
        ["Scaffold_2", "1", "10", "1", "W", "H2.scaffold_1", "1", "10", "+", "Painted", "Hap_2"],
        # Record 7: dummy values (e.g., 11-20)
        ["Scaffold_2", "11", "20", "2", "U", "100", "scaffold", "yes", "proximity_ligation", "", ""],
        # Record 8: dummy values (e.g., 21-30)
        ["Scaffold_2", "21", "30", "3", "W", "H2.scaffold_1", "11", "20", "-", "Painted", ""],
        # Record 9: dummy values (e.g., 31-40)
        ["Scaffold_2", "31", "40", "4", "U", "100", "scaffold", "yes", "proximity_ligation", "", ""],
        # Record 10: dummy values (e.g., 41-50)
        ["Scaffold_2", "41", "50", "5", "W", "H1.scaffold_1", "21", "30", "+", "Painted", ""]
    ]

    # Define the Group ID values for head records:
    group_ids = [
        "Super1_Hap_1",   # Record 1
        "Super1_Hap_1",   # Record 2
        "Super1_Hap_1",   # Record 3
        "Super1_Hap_1",   # Record 4
        "Super1_Hap_1",   # Record 5
        "Super1_Hap_2",   # Record 6
        "Super1_Hap_2",   # Record 7
        "Super1_Hap_2",   # Record 8
        "Super1_Hap_2",   # Record 9
        "Super_2_Hap_1_Unloc_1"  # Record 10
    ]

    # For each head record, pad with 3 "NaN" values (columns 12, 13, 14) and then add the Group ID as the 15th column.
    for i, rec in enumerate(head_records):
        padded_record = rec + ["NaN", "NaN", "NaN"] + [group_ids[i]]
        f.write("\t".join(padded_record) + "\n")

    # --- Tail Records (3 records) ---
    # Tail records remain unchanged.
    tail_records = [
        "Scaffold_600\t1\t10\t1\tW\tH1.scaffold_1099\t1\t10\t+",
        "Scaffold_601\t1\t10\t1\tW\tH1.scaffold_1132\t1\t10\t+",
        "Scaffold_609\t1\t10\t1\tW\tH2.scaffold_542\t1\t10\t+"
    ]

    for rec in tail_records:
        f.write(rec + "\n")

print(f"New AGP file '{agp_filename}' generated.\n")

# -------------------------------
# Part 2: Generate a dummy FASTA file (dummy.fasta)
# -------------------------------

fasta_filename = "dummy.fasta"
with open(fasta_filename, "w") as f:
    f.write(">H1.scaffold_1\n")
    f.write("AAACCCGGGGGGAAAAAAGGGGGGGGAAAAACCCCGACGACGACGACCCGGGAAACCGGGAACCCGGGGCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG\n")

    f.write(">H2.scaffold_1\n")
    f.write("GGGCCCAAAAAAAAAAACCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\n")

    f.write(">H1.scaffold_1099\n")
    f.write("A" * 12 + "\n")

    f.write(">H1.scaffold_1132\n")
    f.write("G" * 10 + "\n")

    f.write(">H2.scaffold_542\n")
    f.write("CCCGGGAAAAAAAAAAGCAGCAGCAGCGAGCGACGACGA" + "\n")

print(f"Dummy FASTA file '{fasta_filename}' generated.\n")

# -------------------------------
# Part 3: Print contents of both files
# -------------------------------

print("Contents of new_dummy.agp:\n")
with open(agp_filename, "r") as f:
    print(f.read())

print("\nContents of dummy.fasta:\n")
with open(fasta_filename, "r") as f:
    print(f.read())


New AGP file 'new_dummy.agp' generated.

Dummy FASTA file 'dummy.fasta' generated.

Contents of new_dummy.agp:

# Object	Object_Begin	Object_End	Part_Number	Component_Type	Component_ID	Component_Begin	Component_End	Orientation	Comment	Extra_info	Col12	Col13	Col14	Group ID
Scaffold_1	1	10	1	W	H1.scaffold_1	1	10	+	Painted	Hap_1	NaN	NaN	NaN	Super1_Hap_1
Scaffold_1	11	20	2	U	100	scaffold	yes	proximity_ligation			NaN	NaN	NaN	Super1_Hap_1
Scaffold_1	11	20	3	W	H1.scaffold_1	11	20	-	Painted		NaN	NaN	NaN	Super1_Hap_1
Scaffold_1	21	30	4	U	100	scaffold	yes	proximity_ligation			NaN	NaN	NaN	Super1_Hap_1
Scaffold_1	21	30	5	W	H1.scaffold_1	21	30	+	Painted		NaN	NaN	NaN	Super1_Hap_1
Scaffold_2	1	10	1	W	H2.scaffold_1	1	10	+	Painted	Hap_2	NaN	NaN	NaN	Super1_Hap_2
Scaffold_2	11	20	2	U	100	scaffold	yes	proximity_ligation			NaN	NaN	NaN	Super1_Hap_2
Scaffold_2	21	30	3	W	H2.scaffold_1	11	20	-	Painted		NaN	NaN	NaN	Super1_Hap_2
Scaffold_2	31	40	4	U	100	scaffold	yes	proximity_ligation			NaN	NaN	NaN	Super1_Hap_2


In [13]:
import pandas as pd
df = pd.read_csv('new_dummy.agp',
                 sep='\t',
                 comment="#",
                 names = [
                    "Object", "Object_begin", "Object_end", "Part_number",
                    "Component_type", "Component_id", "Component_begin",
                    "Component_end", "Orientation","mod","paint","hap","ex1","ex2","Group_ID"
                    ])


In [14]:
import os
import pandas as pd
from Bio import SeqIO

# Create output folders if they don't exist.
os.makedirs("hap_1", exist_ok=True)
os.makedirs("hap_2", exist_ok=True)

# Load the FASTA file as a dictionary.
fasta_file = "dummy.fasta"  # your FASTA file name
fasta_dict = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))

# Assume your DataFrame is loaded as df.
# For example:
# df = pd.read_csv("data.csv", header=None)

# Fixed output file names.
output_file_h1 = os.path.join("hap_1", "Hap_1.fasta")
output_file_h2 = os.path.join("hap_2", "Hap_2.fasta")

# Create empty output files.
open(output_file_h1, "w").close()
open(output_file_h2, "w").close()

# We'll track the last header written for each output file.
#last_header = {"hap_1": None, "hap_2": None}
header=None
# Process each row.
for idx, row in df.iterrows():
    # --- Decide output file based solely on row[5] ---
    # Use row[5].strip() to remove leading/trailing spaces.
    rec_key = str(row[5]).strip()

    # Decide output file: if rec_key contains "H2", use hap_2; if it contains "H1", use hap_1.
    if "H2" in rec_key:
        current_path = output_file_h2
        record = fasta_dict.get(rec_key)
    elif "H1" in rec_key:
        current_path = output_file_h1
        record = fasta_dict.get(rec_key)
    else:
        with open(current_path, "a") as outf:
            outf.write("N" * 100)


    try:
        start_idx = int(row[6])
        end_idx = int(row[7])
    except:
        continue

    sign = str(row[8]).strip()
    if sign == '+':
        seq_str = str(record.seq[start_idx - 1: end_idx])
    elif sign == '-':
        seq_str = str(record.seq[end_idx: start_idx-1:-1])
    else:
        print(f"Unrecognized strand '{sign}' on row {idx}.")
        continue


    # --- Determine header from row[14] (if provided) ---
    if len(row) > 14 and pd.notna(row[14])and header != str(row[14]) :
      header = str(row[14])
      # Remove the substring if present.
      if "_Hap_1" in header:
        with open(current_path, "a") as outf:
          outf.write("\n")
          processed_header = header.replace("_Hap_1", "")
          outf.write(f">{processed_header}\n")
          outf.write(seq_str)
      elif "_Hap_2" in header:
        with open(current_path, "a") as outf:
          outf.write("\n")
          processed_header = header.replace("_Hap_2", "")
          outf.write(f">{processed_header}\n")
          outf.write(seq_str)
    elif header == str(row[14]) and rec_key!='100':
      with open(current_path, "a") as outf:
        outf.write(seq_str)
    elif pd.isna(row[14]):
      with open(current_path, "a") as outf:
        outf.write("\n")
        outf.write(str(row[5]))
        outf.write("\n")
        outf.write(seq_str)


# Print contents.
print("Contents of Hap_1.fasta:")
with open(output_file_h1, "r") as f:
    print(f.read())

print("\nContents of Hap_2.fasta:")
with open(output_file_h2, "r") as f:
    print(f.read())





Contents of Hap_1.fasta:

>Super1
AAACCCGGGGNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGGGAAAAAAGNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGGGGGGAAAA
>Super_2_Unloc_1
GGGGGGAAAA
H1.scaffold_1099
AAAAAAAAAA
H1.scaffold_1132
GGGGGGGGGG

Contents of Hap_2.fasta:

>Super1
GGGCCCAAAANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNCCCCAAAAAANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
H2.scaffold_542
CCCGGGAAAA


/tmp/ipykernel_160496/1078711554.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rec_key = str(row[5]).strip()
/tmp/ipykernel_160496/1078711554.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  start_idx = int(row[6])
/tmp/ipykernel_160496/1078711554.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  end_idx = int(row[7])
/tmp/ipykernel_160496/1078711554.py:52: FutureWarning: Series.__getitem__ treating keys as positions i

In [ ]:
df